In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
house_precinct_csv = 'data/raw/HOUSE_precinct_general.csv'
dime_csv = 'data/raw/dime.csv'
census_mi_vtd_csv = 'data/MI_VTD_demographics.csv'
census_house_mi_csv = 'data/mi_match_vtd_house.csv'

In [3]:
#HOUSE official dtypes from documentation
official_dtypes = {'precinct':str,'office':str, 'party_detailed':str, 'party_simplified':str,
'mode':str,'votes':int, 'county_name':str, 'county_fips':str, 'jurisdiction_name':str,
'jurisdiction_fips':str, 'candidate':str, 'district':str, 'dataverse':str,'year':int,
'stage':str, 'state':str, 'special':str, 'writein':str, 'state_po':str, 'state_fips':str,
'state_cen':str, 'state_ic':str, 'date':str, 'readme_check':str,'magnitude':int}

house =  pd.read_csv(house_precinct_csv, dtype = official_dtypes)

In [4]:
#Filter states
states = ['MI']
house = house[house['state_po'].isin(states)]

In [5]:
#Filter office to US House only
house = house[house['office'].isin(['US HOUSE'])]

In [6]:
#Filter precincts with zero votes
house = house[house['votes']!=0]

In [7]:
#Filter Statistical Adjustments
house = house[house['jurisdiction_name']!="{STATISTICAL ADJUSTMENTS}"]

In [8]:
# Michigan also reports the 'absentee' vote (mode == 'ABSENTEE')
# Is unclear how to match the absentee ballot votes to a precinct
# For this reason, I am dropping them here.
# We should check how this might affect our analysis.
house = house[house['mode']!='ABSENTEE']

In [9]:
#Concatenate state and district to match district column in DIME
house['state_po_district'] = house['state_po'] + house['district'].str[1:]

In [10]:
#Each state reports precincts differently
#The following is only for MI
house[['precinct_code', 'ward_code']] = house['precinct'].str.split(',', expand = True)
house['precinct_code'] = house['precinct_code'].apply('{0:0>3}'.format)
house['ward_code'] = house['ward_code'].str.replace('WARD','')
house['ward_code'] = house['ward_code'].str.strip()
house['ward_code'] = house['ward_code'].fillna('0')
house['ward_code'] = house['ward_code'].apply('{0:0>2}'.format)

In [11]:
#Note: We still need to check the precincts that have code with letters like '0-22L', '1-A', '1-B' for MI
#Dropping them from now
house['tocheck'] = np.where(house.precinct_code.str.contains('-'), '1', '0')
house['precinct'][(house['tocheck'] == '1')]
house = house[house['tocheck']!='1']

In [12]:
#Create NAME20 to match census data
#In census data NAME20 = COUNTYFIPS + MCDFIPS + WARD + PRECINCT
#In house data we have jurisdiction_fips = state_fips + county_fips + mcd_fips
#and we created ward_code and precinct_code
#so we have to remove state_fips from jurisdiction_fips and add ward_code and precinct_code
house['NAME20'] = house['jurisdiction_fips'].str[2:] + house['ward_code'] + house['precinct_code']

In [13]:
#Edit candidate names based on inspection
house['candidate']  = house['candidate'] .str.replace('FRED UPTON','FREDERICK STEPHEN UPTON')

In [14]:
#Totals by precinct for MI
total_by_prec_cand = house.groupby(['state_po', 'state_po_district', 'NAME20','candidate'])['votes'].sum().reset_index()
total_by_prec_cand


,state_po,state_po_district,NAME20,candidate,votes
0,MI,MI01,0010104000001,JACK BERGMAN,449
1,MI,MI01,0010104000001,MATTHEW W MORGAN,214
2,MI,MI01,0011246000001,JACK BERGMAN,395
3,MI,MI01,0011246000001,MATTHEW W MORGAN,181
4,MI,MI01,0011932000001,JACK BERGMAN,365
...,...,...,...,...,...
15315,MI,MI14,1638245300001,MARC S HERSCHFUS,681
15316,MI,MI14,1638245300001,PHILIP KOLODY,8
15317,MI,MI14,1638245300002,BRENDA LAWRENCE,221
15318,MI,MI14,1638245300002,MARC S HERSCHFUS,410


In [15]:
total_by_prec = total_by_prec_cand.groupby(['state_po', 'state_po_district', 'NAME20'])['votes'].sum().reset_index()
total_by_prec = total_by_prec.rename(columns = {'votes': 'total_votes'})
total_by_prec

,state_po,state_po_district,NAME20,total_votes
0,MI,MI01,0010104000001,663
1,MI,MI01,0011246000001,576
2,MI,MI01,0011932000001,562
3,MI,MI01,0013482000001,675
4,MI,MI01,0013574000001,302
...,...,...,...,...
4747,MI,MI14,1633670000004,954
4748,MI,MI14,1633670000005,1376
4749,MI,MI14,1633670000006,1195
4750,MI,MI14,1638245300001,968


In [16]:
result = pd.merge(total_by_prec_cand, total_by_prec, on = ['state_po', 'state_po_district', 'NAME20'])
result['pct_vote'] = result['votes']/result['total_votes']
result['candidate_house'] = result['candidate'].str.lower()
result['candidate_house'] = result['candidate_house'].str.replace('.', '')
result['candidate_house'] = result['candidate_house'].str.strip()
result

/var/folders/sn/cp78km453yq1y8g3c822ww7r0000gn/T/ipykernel_35736/2228020900.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result['candidate_house'] = result['candidate_house'].str.replace('.', '')


,state_po,state_po_district,NAME20,candidate,votes,total_votes,pct_vote,candidate_house
0,MI,MI01,0010104000001,JACK BERGMAN,449,663,0.677225,jack bergman
1,MI,MI01,0010104000001,MATTHEW W MORGAN,214,663,0.322775,matthew w morgan
2,MI,MI01,0011246000001,JACK BERGMAN,395,576,0.685764,jack bergman
3,MI,MI01,0011246000001,MATTHEW W MORGAN,181,576,0.314236,matthew w morgan
4,MI,MI01,0011932000001,JACK BERGMAN,365,562,0.649466,jack bergman
...,...,...,...,...,...,...,...,...
15315,MI,MI14,1638245300001,MARC S HERSCHFUS,681,968,0.703512,marc s herschfus
15316,MI,MI14,1638245300001,PHILIP KOLODY,8,968,0.008264,philip kolody
15317,MI,MI14,1638245300002,BRENDA LAWRENCE,221,636,0.347484,brenda lawrence
15318,MI,MI14,1638245300002,MARC S HERSCHFUS,410,636,0.644654,marc s herschfus


In [17]:
#Commenting the lines where we computed the total by county for the second deliverable
#Totals by county
#total_by_county_cand = house.groupby(['state_po', 'state_po_district', 'county_fips','candidate'])['votes'].sum().reset_index()
#total_by_county = total_by_county_cand.groupby(['state_po', 'state_po_district', 'county_fips'])['votes'].sum().reset_index()
#total_by_county = total_by_county.rename(columns = {'votes': 'total_votes'})

In [18]:
#result = pd.merge(total_by_county_cand, total_by_county, on = ['state_po', 'state_po_district', 'county_fips'])
#result['pct_vote'] = result['votes']/result['total_votes']
#result['candidate_house'] = result['candidate'].str.lower()
#result['candidate_house'] = result['candidate_house'].str.replace('.', '')
#result['candidate_house'] = result['candidate_house'].str.strip()

In [19]:
#DIME
dime =  pd.read_csv(dime_csv)
dime.sort_values(by=['district', 'name'])
dime = dime.drop(columns=['cycle', 'seat', 'ran_general', 'winner', 'recipient_type'])

In [20]:
# new data frame with split name
split = dime['name'].str.split(",", n = 1, expand = True)

In [21]:
# Change party number codes to letter codes
dime.loc[dime['party'] == 100, 'party'] = 'D'
dime.loc[dime['party'] == 200, 'party'] = 'R'
dime.loc[dime['party'] == 328, 'party'] = 'I'

In [22]:
#Create new candidate name to match with HOUSE df, and new id column to match with census
dime['candidate_dime'] = dime['ffname'] + " " + split[0]
dime['candidate_dime'] = dime['candidate_dime'].str.replace('.', '')
dime['candidate_dime'] = dime['candidate_dime'].str.replace('mr', '')
dime['candidate_dime'] = dime['candidate_dime'].str.strip()

dime['census_match'] = dime['party'] + dime['name'].astype(str).str[:3] + dime['state']

/var/folders/sn/cp78km453yq1y8g3c822ww7r0000gn/T/ipykernel_35736/138490475.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dime['candidate_dime'] = dime['candidate_dime'].str.replace('.', '')


In [23]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [24]:
result_MI = result[result['state_po'].isin(['MI'])]
result_MI = result_MI['candidate_house'].drop_duplicates()
dime_MI = dime[dime['state'].isin(['MI'])]

str2Match = result_MI.tolist()
strOptions = dime_MI['candidate_dime'].fillna('######').tolist()

name_match,ratio_match=checker(str2Match,strOptions)
merged_MI = pd.DataFrame()
merged_MI['candidate_house'] = pd.Series(str2Match)
merged_MI['candidate_dime'] = pd.Series(name_match)
merged_MI['correct_ratio'] = pd.Series(ratio_match).astype(float)

merged_MI.sort_values(by=['correct_ratio'])

,candidate_house,candidate_dime,correct_ratio
41,marc joseph sosnowski,jeremy michael peruski,42.0
40,d etta wilcoxon,weldon frederick wooden,42.0
46,philip kolody,william r wild,44.0
7,ted gerrard,leonard schwartz,44.0
45,marc s herschfus,candius stearns,45.0
4,ronald e graeser,paul colin clements,46.0
12,kathy goodwin,matt longjohn,46.0
23,andrea kirby,rashida tlaib,48.0
8,joe farrington,weldon frederick wooden,49.0
36,gary walkowicz,a rocky raczkowski,50.0


In [25]:
#merge with pivots of fuzzy matching
result = result.merge(merged_MI, how = 'left', on = 'candidate_house')
result

,state_po,state_po_district,NAME20,candidate,votes,total_votes,pct_vote,candidate_house,candidate_dime,correct_ratio
0,MI,MI01,0010104000001,JACK BERGMAN,449,663,0.677225,jack bergman,john bergman,75.0
1,MI,MI01,0010104000001,MATTHEW W MORGAN,214,663,0.322775,matthew w morgan,matthew wade morgan,93.0
2,MI,MI01,0011246000001,JACK BERGMAN,395,576,0.685764,jack bergman,john bergman,75.0
3,MI,MI01,0011246000001,MATTHEW W MORGAN,181,576,0.314236,matthew w morgan,matthew wade morgan,93.0
4,MI,MI01,0011932000001,JACK BERGMAN,365,562,0.649466,jack bergman,john bergman,75.0
...,...,...,...,...,...,...,...,...,...,...
15315,MI,MI14,1638245300001,MARC S HERSCHFUS,681,968,0.703512,marc s herschfus,candius stearns,45.0
15316,MI,MI14,1638245300001,PHILIP KOLODY,8,968,0.008264,philip kolody,william r wild,44.0
15317,MI,MI14,1638245300002,BRENDA LAWRENCE,221,636,0.347484,brenda lawrence,brenda lulenar lawrence,100.0
15318,MI,MI14,1638245300002,MARC S HERSCHFUS,410,636,0.644654,marc s herschfus,candius stearns,45.0


In [26]:
# merge with dime data to add ideology
result = result.merge(dime, how = 'left', on = 'candidate_dime')
result

,state_po,state_po_district,NAME20,candidate,votes,total_votes,pct_vote,candidate_house,candidate_dime,correct_ratio,...,bonica_rid,name,ffname,party,state,district,recipient_cfscore,recipient_cfscore_dyn,district_partisanship,census_match
0,MI,MI01,0010104000001,JACK BERGMAN,449,663,0.677225,jack bergman,john bergman,75.0,...,cand137055,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI
1,MI,MI01,0010104000001,MATTHEW W MORGAN,214,663,0.322775,matthew w morgan,matthew wade morgan,93.0,...,cand140638,"morgan, matthew wade",matthew wade,D,MI,MI01,-1.474,-1.318,0.233,DmorMI
2,MI,MI01,0011246000001,JACK BERGMAN,395,576,0.685764,jack bergman,john bergman,75.0,...,cand137055,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI
3,MI,MI01,0011246000001,MATTHEW W MORGAN,181,576,0.314236,matthew w morgan,matthew wade morgan,93.0,...,cand140638,"morgan, matthew wade",matthew wade,D,MI,MI01,-1.474,-1.318,0.233,DmorMI
4,MI,MI01,0011932000001,JACK BERGMAN,365,562,0.649466,jack bergman,john bergman,75.0,...,cand137055,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15315,MI,MI14,1638245300001,MARC S HERSCHFUS,681,968,0.703512,marc s herschfus,candius stearns,45.0,...,cand142531,"stearns, candius mrs.",candius .,R,MI,MI09,1.179,1.169,-0.021,RsteMI
15316,MI,MI14,1638245300001,PHILIP KOLODY,8,968,0.008264,philip kolody,william r wild,44.0,...,cand144684,"wild, william r.",william r.,D,MI,MI13,0.023,0.147,-2.429,DwilMI
15317,MI,MI14,1638245300002,BRENDA LAWRENCE,221,636,0.347484,brenda lawrence,brenda lulenar lawrence,100.0,...,cand42832,"lawrence, brenda lulenar",brenda lulenar,D,MI,MI14,-0.709,-0.684,-2.671,DlawMI
15318,MI,MI14,1638245300002,MARC S HERSCHFUS,410,636,0.644654,marc s herschfus,candius stearns,45.0,...,cand142531,"stearns, candius mrs.",candius .,R,MI,MI09,1.179,1.169,-0.021,RsteMI


In [27]:
# Calculate the min correct ratio by precinct
result = result.join(result.groupby(['state_po_district', 'NAME20'])['correct_ratio'].agg(['min']), on = ['state_po_district', 'NAME20'])

# Keep only precincts where min correct ratio is at least 70
result = result[result['min']>=70]
result


,state_po,state_po_district,NAME20,candidate,votes,total_votes,pct_vote,candidate_house,candidate_dime,correct_ratio,...,name,ffname,party,state,district,recipient_cfscore,recipient_cfscore_dyn,district_partisanship,census_match,min
0,MI,MI01,0010104000001,JACK BERGMAN,449,663,0.677225,jack bergman,john bergman,75.0,...,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI,75.0
1,MI,MI01,0010104000001,MATTHEW W MORGAN,214,663,0.322775,matthew w morgan,matthew wade morgan,93.0,...,"morgan, matthew wade",matthew wade,D,MI,MI01,-1.474,-1.318,0.233,DmorMI,75.0
2,MI,MI01,0011246000001,JACK BERGMAN,395,576,0.685764,jack bergman,john bergman,75.0,...,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI,75.0
3,MI,MI01,0011246000001,MATTHEW W MORGAN,181,576,0.314236,matthew w morgan,matthew wade morgan,93.0,...,"morgan, matthew wade",matthew wade,D,MI,MI01,-1.474,-1.318,0.233,DmorMI,75.0
4,MI,MI01,0011932000001,JACK BERGMAN,365,562,0.649466,jack bergman,john bergman,75.0,...,"bergman, john",john,R,MI,MI01,1.086,1.044,0.233,RberMI,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13395,MI,MI13,1632200000454,BRENDA JONES,2,4,0.500000,brenda jones,brenda jones,100.0,...,"jones, brenda",brenda,D,MI,MI13,-0.559,-0.376,-2.429,DjonMI,100.0
13396,MI,MI13,1632200000454,RASHIDA TLAIB,2,4,0.500000,rashida tlaib,rashida tlaib,100.0,...,"tlaib, rashida",rashida,D,MI,MI13,-1.378,-1.174,-2.429,DtlaMI,100.0
13411,MI,MI13,1632200000458,BRENDA JONES,5,10,0.500000,brenda jones,brenda jones,100.0,...,"jones, brenda",brenda,D,MI,MI13,-0.559,-0.376,-2.429,DjonMI,100.0
13412,MI,MI13,1632200000458,RASHIDA TLAIB,5,10,0.500000,rashida tlaib,rashida tlaib,100.0,...,"tlaib, rashida",rashida,D,MI,MI13,-1.378,-1.174,-2.429,DtlaMI,100.0


In [29]:
#Pivot data from long to wide
result = result[['state_po_district', 'NAME20', 'candidate_house',
                'candidate_dime', 'pct_vote', 'party',
                'recipient_cfscore']]

KeyError: "['wgt_cfscore', 'cf_category', 'cf_label'] not in index"

In [ ]:
pivot = result.pivot_table(index=['NAME20'], 
                            columns=['party'])
pivot.columns = ["_".join((j,i)) for i,j in pivot.columns]
pivot

In [ ]:
# Compute ideology score weighted by % votes
#result['wgt_cfscore'] = result['recipient_cfscore'] * result['pct_vote']
#result

In [ ]:
for x in ['D', 'I', 'R']:
    score = x + '_recipient_cfscore'
    cat = x + '_cf_category'
    label = x + '_cf_label'

    pivot[cat] = ''
    pivot.loc[(pivot[score] < -1), cat] = '-3'
    pivot.loc[(pivot[score] >= -1) & (pivot[score] <= -0.5), cat] = '-2'
    pivot.loc[(pivot[score] >= -0.5) & (pivot[score] < 0), cat] = '-1'
    pivot.loc[(pivot[score] >= 0) & (pivot[score] < 0.5), cat] = '1'
    pivot.loc[(pivot[score] >= 0.5) & (pivot[score] < 1), cat] = '2'
    pivot.loc[(pivot[score] >= 1), cat] = '3'

    pivot[label] = ''
    pivot.loc[(pivot[cat] == '-3'), label] = 'Very Liberal'
    pivot.loc[(pivot[cat] == '-2'), label] = 'Strong Liberal'
    pivot.loc[(pivot[cat] == '-1'), label] = 'Lean Liberal'
    pivot.loc[(pivot[cat] == '1'), label] = 'Lean Conservative'
    pivot.loc[(pivot[cat] == '2'), label] = 'Strong Conservative'
    pivot.loc[(pivot[cat] == '3'), label] = 'Very Conservative'
    
pivot

In [ ]:
#Map Census to House data
map_census_house = pd.read_csv(census_house_mi_csv, dtype = {'GEOID20':str, 'NAME20':str, 'jurisdiction_fips':str})
map_census_house_mi = map_census_house[['GEOID20', 'NAME20']]
map_census_house_mi['VTD_ID'] = map_census_house_mi['GEOID20']

In [ ]:
mi_result = pivot.merge(map_census_house_mi, how = 'left', on = 'NAME20')

In [ ]:
mi_result

In [ ]:
#Census data

mi_vtd = pd.read_csv(census_mi_vtd_csv)
# Drop rows where VTD has ZZZ
remove = mi_vtd['VTD_ID'].str.contains('Z')
mi_vtd = mi_vtd[ ~remove]
mi_vtd

In [ ]:
mi_result = mi_result.merge(mi_vtd, how = 'left', on = 'VTD_ID')
mi_result

In [ ]:
mi_result.columns

In [ ]:
mi_result = mi_result[['NAME20','VTD_ID', 'total_pop',
       'pop_density', 'prop_white', 'prop_Black', 'prop_Hispanic', 'prop_AIA',
       'prop_Asian', 'prop_married', 'prop_nonfamily',
       'prop_family_nonmarried', 'prop_hs_diploma', 'prop_some_college',
       'prop_bachelors', 'prop_higher', 'prop_under_hs', 'prop_under_30k',
       'prop_30_to_50k', 'prop_50_to_100k', 'prop_more_than_100k',
       'D_pct_vote', 'I_pct_vote', 'R_pct_vote',
       'D_recipient_cfscore', 'I_recipient_cfscore', 'R_recipient_cfscore',
       'D_cf_category', 'D_cf_label', 'I_cf_category', 'I_cf_label',
       'R_cf_category', 'R_cf_label']]

In [ ]:
#Export to csv
mi_result.to_csv('data/mi_matched_precinct.csv', index = False)

In [ ]:
#result_agg = result.groupby(['state_po_district', 'county_fips'])['wgt_cfscore'].sum().reset_index()
#result_agg

In [ ]:
#Lean Liberal (0 through -0.5)
#Strong Liberal (-0.5 through -1)
#Very Liberal (-1 and lower)
#Lean Conservative (0 through 0.5)
#Strong Conservative (0.5 through 1)
#Very Conservative (1 and higher)
# result_agg['cf_category'] = ''
# result_agg.loc[(result_agg['wgt_cfscore'] < -1), 'cf_category'] = '-3'
# result_agg.loc[(result_agg['wgt_cfscore'] >= -1) & (result_agg['wgt_cfscore'] <= -0.5), 'cf_category'] = '-2'
# result_agg.loc[(result_agg['wgt_cfscore'] >= -0.5) & (result_agg['wgt_cfscore'] < 0), 'cf_category'] = '-1'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 0) & (result_agg['wgt_cfscore'] < 0.5), 'cf_category'] = '1'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 0.5) & (result_agg['wgt_cfscore'] < 1), 'cf_category'] = '2'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 1), 'cf_category'] = '3'

# result_agg['cf_label'] = ''
# result_agg.loc[(result_agg['cf_category'] == '-3'), 'cf_label'] = 'Very Liberal'
# result_agg.loc[(result_agg['cf_category'] == '-2'), 'cf_label'] = 'Strong Liberal'
# result_agg.loc[(result_agg['cf_category'] == '-1'), 'cf_label'] = 'Lean Liberal'
# result_agg.loc[(result_agg['cf_category'] == '1'), 'cf_label'] = 'Lean Conservative'
# result_agg.loc[(result_agg['cf_category'] == '2'), 'cf_label'] = 'Strong Conservative'
# result_agg.loc[(result_agg['cf_category'] == '3'), 'cf_label'] = 'Very Conservative'

# result_agg

In [ ]:
#export to csv
#result_agg.to_csv('data/ideology_county.csv', index = False)

In [ ]:
#mi_result.dropna(inplace=True)
#export to csv
#mi_result.to_csv('data/mi_matched_county.csv', index = False)

In [ ]:
# for x in ['prop_white', 'prop_Black', 'prop_Hispanic', 'prop_AIA', 'prop_Asian']:
#     mi_vtd[x] = mi_vtd['total_pop'] * mi_vtd[x]
# mi_vtd

In [ ]:
# mi_vtd['county_fips'] = mi_vtd['VTD_ID'].str[:5]
# mi_vtd

In [ ]:
# mi_county = mi_vtd.groupby(['county_fips']).agg(
#      total_pop = ('total_pop','sum'),
#      total_White = ('prop_white','sum'),
#      total_Black = ('prop_Black','sum'),
#      total_Hispanic = ('prop_Hispanic','sum'),
#      total_AIA = ('prop_AIA','sum'),
#      total_Asian = ('prop_Asian','sum'),
#      ).reset_index()
# mi_county

In [ ]:
# mi_county['prop_White'] = mi_county['total_White'] / mi_county['total_pop']
# mi_county['prop_Black'] = mi_county['total_Black'] / mi_county['total_pop']
# mi_county['prop_Hispanic'] = mi_county['total_Hispanic'] / mi_county['total_pop']
# mi_county['prop_AIA'] = mi_county['total_AIA'] / mi_county['total_pop']
# mi_county['prop_Asian'] = mi_county['total_Asian'] / mi_county['total_pop']
# mi_county

In [ ]:
# mi_result = mi_county.merge(result_agg, how = 'left', on = 'county_fips')
# mi_result

In [ ]:
#mi_result = mi_result[['county_fips','total_pop','prop_White','prop_Black','prop_Hispanic','prop_AIA','prop_Asian', 'wgt_cfscore','cf_category','cf_label']]
